In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime


plt.style.use("fivethirtyeight")

In [ ]:
%%time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/data.csv')

print()
print('Train')
display(train.head(3))



print('Dataframe shapes:', train.shape)
print()

submission = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv',index_col='row-col')
print()
print('Submission')
display(submission.head(3))
print('Dataframe shapes:', submission.shape)

In [ ]:
'''train.info()'''

In [ ]:
'''train.drop('row_id', inplace=True, axis=1)
train.describe().T.sort_values(by='mean' , ascending = False)\
.style.background_gradient(cmap="Greys")\
.bar(subset=["mean"],color='#FBB03B')\
.bar(subset=["max"],color='#ff355d')'''


In [ ]:
'''print("Null Values in Training Set:")
null_values=train.isna().sum()
print(null_values[1:15])
print(null_values[41:80])'''

In [ ]:
'''print("Duplicate Values in Training Set:")
display(train.duplicated().sum())'''

In [ ]:
'''int_features = list(train.select_dtypes(include='int').columns)
int_features.remove('row_id')
float_features = list(train.select_dtypes(include='float').columns)

print("int featres:", int_features)
print("float featres:", float_features)
'''

In [ ]:
'''print("int_features in Training Set:")
display(train[int_features].nunique())'''

In [ ]:
'''from IPython.core.display import HTML
def value_counts_all(df, columns):
    pd.set_option('display.max_rows', 50)
    table_list = []
    for col in columns:
        table_list.append(pd.DataFrame(df[col].value_counts()))
    return HTML(
        f"<table><tr> {''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list])} </tr></table>")'''

In [ ]:
'''%%time
value_counts_all(train, int_features)'''

In [ ]:
'''%%time
import missingno as msno
msno.matrix(train)'''

In [ ]:
'''!git clone https://github.com/analokmaus/kuma_utils.git'''

In [ ]:
'''import sys
sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer'''

In [ ]:
'''%%time
lgbm_imtr = LGBMImputer(n_iter=100, verbose=True)

train[:] = lgbm_imtr.fit_transform(train)'''

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBRegressor

In [ ]:
%%time
xgb_imputer=IterativeImputer(estimator=XGBRegressor(n_estimators=999,
                                                    tree_method='gpu_hist',
                                                    predictor="gpu_predictor",
                                                    n_jobs=-1),verbose=3,max_iter=3)
train[:] = xgb_imputer.fit_transform(train)

In [ ]:
'''%%time
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

imp = KNNImputer()

train[:] = imp.fit_transform(train)'''

In [ ]:
%%time
from tqdm import tqdm
for i in tqdm(submission.index):
    row = int(i.split('-')[0])
    col = i.split('-')[1]
    submission.loc[i, 'value'] = train.loc[row, col]

submission.to_csv('submission.csv')
submission